# 数据历史同步

In [32]:
import pandas as pd
import datetime as dtt

class history_compare:
    def __init__(self,start,end,data_type):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.start = start
        self.end = end
        self.his_type = {
            "rain":"rain.csv",
            "wind":"wind.csv",
            "sun":"sun.csv",
            "temp":"temp.csv"}
        self.data_type = self.his_type[data_type]
    def read_his_csv(self):
        data = pd.read_csv(self.data_type)
        data = data[data['City']=="台州市"]
        return data
    def read_normal(self):
        data = pd.read_csv("2019.csv")
        data = data[(data['Datetime']>=self.start)&(data['Datetime']<=self.end)]
        data = data[data['City']=="台州市"]
        return data
    def get_history(self):
        his = self.read_his_csv()
        start_year = int(self.start[0:4])
        end_year = int(self.start[0:4])
        # 获取开始日期的日期index
        if start_year % 4 == 0 and (start_year % 100 != 0 or start_year % 400 == 0):
            start_index = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday
            if start_index>=60:
                start_index = start_index -1
            else:
                start_index = start_index
        else:
            start_index = dtt.datetime.strptime(self.start, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday
        # 获取结束日期的
        if end_year % 4 == 0 and (end_year % 100 != 0 or end_year % 400 == 0):
            end_index = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday
            if end_index>=60:
                end_index = end_index -1
            else:
                end_index = end_index
        else:
            end_index = dtt.datetime.strptime(self.end, "%Y-%m-%d %H:%M:%S").timetuple().tm_yday
        hiscut = his[(his['Day_Seq']>=start_index)&(his['Day_Seq']<=end_index)]
        return hiscut
    def compart_rain(self):
        his = self.get_history()
        now = self.read_normal()
        his = his[(his['PRE']>0)&(his['PRE']<20000)]
        now = now[(now['PRE_Time_2020']>0)&(now['PRE_Time_2020']<20000)]
        his_group = his.groupby(["Station_Id_C",'Station_Name','City','Lat','Lon','Alti'])['PRE'].sum().reset_index()
        now_group = now.groupby(["Station_Id_C"])['PRE_Time_2020'].sum().reset_index()
        all_data = his_group.join(now_group, lsuffix="_left", rsuffix="_right")
#         da = now_group.join(his_group, lsuffix="_left", rsuffix="_right")
        return all_data

start = '2019-08-09 00:00:00'
end = '2019-08-10 00:00:00'
data_type = 'rain'
worker = history_compare(start,end,data_type)
# data = worker.timeind()
# data = worker.get_history()
worker.compart_rain()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.


,Station_Id_C_left,Station_Name,City,Lat,Lon,Alti,PRE,Station_Id_C_right,PRE_Time_2020
0,58559,天台,台州市,29.1528,120.9706,107.6,26.5,58559,228.7
1,58568,三门,台州市,29.1228,121.3833,34.5,55.7,58568,337.2
2,58652,仙居,台州市,28.8672,120.7189,83.2,30.0,58652,202.1
3,58660,临海,台州市,28.8739,121.1928,6.3,43.0,58653,813.0
4,58664,温岭,台州市,28.3728,121.3589,35.0,38.9,58660,376.4
5,58665,洪家,台州市,28.6181,121.4164,4.6,27.1,58661,185.0
6,58666,大陈,台州市,28.4514,121.9033,89.1,21.4,58662,131.2
7,58667,玉环,台州市,28.0775,121.2708,95.9,28.5,58663,129.0


In [21]:
import pandas as pd

data_1 = pd.DataFrame([[1, 2], [4, 5], [5, 6]], columns=["a", "b"])
data_2 = pd.DataFrame([[7, 8], [11, 12]], columns=["c", "d"])
print("data_1:", data_1)
print("data_2:", data_2)
data_3 = data_1.join(data_2)
print("data_3:", data_3)

data_1:    a  b
0  1  2
1  4  5
2  5  6
data_2:     c   d
0   7   8
1  11  12
data_3:    a  b     c     d
0  1  2   7.0   8.0
1  4  5  11.0  12.0
2  5  6   NaN   NaN


In [5]:
降水比较
温度比较
日照比较
# 降水日数
# 高温日数


,Unnamed: 0,Station_Id_C,City,Town,Station_levl,Station_Name,Lat,Lon,Alti,Day_Seq,PRE
301710,301710,58660,台州市,大洋街道,12,临海,28.8739,121.1928,6.3,221,20.8
301711,301711,58660,台州市,大洋街道,12,临海,28.8739,121.1928,6.3,222,22.2
303535,303535,58559,台州市,始丰街道,12,天台,29.1528,120.9706,107.6,221,12.6
303536,303536,58559,台州市,始丰街道,12,天台,29.1528,120.9706,107.6,222,13.9
306090,306090,58665,台州市,洪家街道,12,洪家,28.6181,121.4164,4.6,221,18.0
306091,306091,58665,台州市,洪家街道,12,洪家,28.6181,121.4164,4.6,222,9.1
631305,631305,58568,台州市,海游街道,12,三门,29.1228,121.3833,34.5,221,31.6
631306,631306,58568,台州市,海游街道,12,三门,29.1228,121.3833,34.5,222,24.1
883155,883155,58652,台州市,安洲街道,12,仙居,28.8672,120.7189,83.2,221,17.0
883156,883156,58652,台州市,安洲街道,12,仙居,28.8672,120.7189,83.2,222,13.0


In [44]:
a = data[(data['Station_Id_C']=="58660")&(data['PRE_Time_2020']<2000)]
a[['PRE_Time_2020','Datetime']]

,PRE_Time_2020,Datetime
5500,156.0,2019-08-09 00:00:00
5520,220.4,2019-08-10 00:00:00


In [29]:
data

,Unnamed: 0,Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,...,WIN_D,WIN_S,WIN_S_Max,WIN_S_Max_OTime,WIN_D_INST_Max,WIN_S_Inst_Max,WIN_S_INST_Max_OTime,GST_Max,GST_Min,SSH
5544,5544,天台县,浙江省,始丰街道,12,天台,台州市,58559,29.1528,120.9706,...,999999,999999.0,5.2,224,282,11.2,223,39.8,24.2,9.7
5545,5545,玉环市,浙江省,坎门街道,12,玉环,台州市,58667,28.0775,121.2708,...,999999,999999.0,12.6,1549,242,16.7,1255,43.6,25.8,8.5
5546,5546,三门县,浙江省,海游街道,12,三门,台州市,58568,29.1228,121.3833,...,999999,999999.0,5.8,118,242,11.6,451,45.1,25.0,10.2
5547,5547,椒江区,浙江省,洪家街道,12,洪家,台州市,58665,28.6181,121.4164,...,999999,999999.0,5.0,1346,228,10.2,1314,44.4,25.2,11.1
5548,5548,仙居县,浙江省,安洲街道,12,仙居,台州市,58652,28.8672,120.7189,...,999999,999999.0,4.6,120,238,7.6,424,44.5,24.2,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9109,9109,临海市,浙江省,上盘镇,16,白沙,台州市,K8611,28.7128,121.6378,...,999999,999999.0,6.0,210,334,11.5,208,999999.0,999999.0,999999.0
9110,9110,天台县,浙江省,南屏乡,16,南屏,台州市,K8706,29.0597,120.9656,...,999999,999999.0,3.6,2217,90,8.5,1819,999999.0,999999.0,999999.0
9111,9111,三门县,浙江省,亭旁镇,16,佃石,台州市,K8818,29.0025,121.3147,...,999999,999999.0,9.3,2239,21,14.3,1721,999999.0,999999.0,999999.0
9112,9112,仙居县,浙江省,湫山乡,16,雅溪,台州市,K8910,28.6981,120.3361,...,999999,999999.0,3.3,605,310,6.5,1307,999999.0,999999.0,999999.0


In [4]:
import pandas as pd
data = pd.read_csv("rain.csv")
data = data[data['City']=="台州市"]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
station = 58667
rain  = data[data['Station_Id_C']==station]
rain[(data['PRE']<588) &(data['PRE']>=0)]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Station_Id_C,City,Town,Station_levl,Station_Name,Lat,Lon,Alti,Day_Seq,PRE
888410,888410,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,295,2.1
888411,888411,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,296,1.3
888412,888412,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,297,0.9
888413,888413,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,298,0.4
888414,888414,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,299,1.8
...,...,...,...,...,...,...,...,...,...,...,...
888770,888770,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,290,0.6
888771,888771,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,291,2.8
888772,888772,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,292,0.0
888773,888773,58667,台州市,坎门街道,12,玉环,28.0775,121.2708,95.9,293,0.3


In [34]:
import pandas as pd
# Index(['Unnamed: 0', 'Cnty', 'Province', 'Town', 'Station_levl',
#        'Station_Name', 'City', 'Station_Id_C', 'Lat', 'Lon', 'Alti',
#        'Datetime', 'TEM_Max', 'TEM_Min', 'VAP_Avg', 'RHU_Avg', 'RHU_Min',
#        'VIS_Min', 'PRE_Max_1h', 'PRE_Time_2008', 'PRE_Time_0820',
#        'PRE_Time_2020', 'PRE_Time_0808', 'WIN_D', 'WIN_S', 'WIN_S_Max',
#        'WIN_S_Max_OTime', 'WIN_D_INST_Max', 'WIN_S_Inst_Max',
#        'WIN_S_INST_Max_OTime', 'GST_Max', 'GST_Min', 'SSH'],
#       dtype='object')
data = pd.read_csv("2019.csv")
station = "58660"
rain  = data[data['Station_Id_C']==station]
# PRE_Time_2020
# rain[(data['PRE_Time_2020']<588) &(data['PRE_Time_2020']>=0)]
rain['Datetime']

1       2019-01-01 00:00:00
27      2019-01-02 00:00:00
51      2019-01-03 00:00:00
82      2019-01-04 00:00:00
101     2019-01-05 00:00:00
               ...         
8995    2019-12-27 00:00:00
9014    2019-12-28 00:00:00
9039    2019-12-29 00:00:00
9070    2019-12-30 00:00:00
9091    2019-12-31 00:00:00
Name: Datetime, Length: 365, dtype: object